## welcome

* Are paid apps getting more popular the free apps? 
* Which category has the highest share? 
* What is the distribution of rating?
* What is the correlation between the size of the app and the cost or her popularity.


In [360]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [361]:
app_df = pd.read_csv("googleplaystore.csv",encoding= "utf-8")
reviews_df = pd.read_csv("googleplaystore_user_reviews.csv")

In [362]:
app_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [363]:
app_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [364]:
app_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [365]:
reviews_df.dtypes

App                        object
Translated_Review          object
Sentiment                  object
Sentiment_Polarity        float64
Sentiment_Subjectivity    float64
dtype: object

In [366]:
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


### <span style="color:red">Data cleaning</span> 
* there are no all-blank rows
* there are many reviews with App names but without any other information 
* change the datatype of "Price" and "Installs" to float
* remove duplicate

In [367]:
print("reviews DataFrame shape before:", reviews_df.shape)
columns = ["Translated_Review","Sentiment","Sentiment_Polarity","Sentiment_Subjectivity"]
null_rows = reviews_df[reviews_df[columns].isna().any(axis=1)]
reviews_df.drop(index=null_rows.index, inplace=True)
print("reviews DataFrame shape after:", reviews_df.shape)

reviews DataFrame shape before: (64295, 5)
reviews DataFrame shape after: (37427, 5)


In [368]:
#there is one row with bed data that i need to remove
bed_row = app_df.loc[app_df["App"]=="Life Made WI-Fi Touchscreen Photo Frame"].index
app_df.drop(index=bed_row, inplace=True)

In [369]:
app_df["Price"] = app_df["Price"].str.replace('$','', regex=True)
app_df["Installs"]= app_df["Installs"].str.replace(r'[\+\,]','', regex=True)

In [370]:
app_df["Price"]= app_df["Price"].astype("float")
app_df["Installs"] = pd.to_numeric(app_df["Installs"])
app_df["Installs"]= app_df["Installs"].astype("int32")

##### Convert to lowercase letters

In [371]:
app_df["App"] = app_df["App"].str.lower()
app_df["Category"] = app_df["Category"].str.lower()
app_df["Genres"] = app_df["Genres"].str.lower()

##### Remove duplicate

In [372]:
print("There are {} duplicates rows in App DataFrame".format(app_df.duplicated(subset=["App"]).sum()))
# print("there are {} duplicates rows", reviews_df.duplicated().sum())

there are 1202 duplicates rows in App DataFrame


In [373]:
app_df.drop_duplicates(subset=["App"],inplace=True)

##### Removing outliers
* I assume that apps that cost more than 70$ are probably a scam. 
* There aren't apps with ratings of more than 5.0

In [374]:
app_df.loc[app_df["Price"]>70.0].head(10)
#"💎 i'm rich app i porbebly a scam"

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2253,vargo anesthesia mega app,medical,4.6,92,32M,1000,Paid,79.99,Everyone,medical,"June 18, 2018",19.0,4.0.3 and up
2390,language therapy: aphasia,medical,NaN,10,28M,1000,Paid,74.99,Everyone,medical,"June 25, 2018",3.1.1,4.0.3 and up
4197,most expensive app (h),family,4.3,6,1.5M,100,Paid,399.99,Everyone,entertainment,"July 16, 2018",1.0,7.0 and up
4362,💎 i'm rich,lifestyle,3.8,718,26M,10000,Paid,399.99,Everyone,lifestyle,"March 11, 2018",1.0.0,4.4 and up
4367,i'm rich - trump edition,lifestyle,3.6,275,7.3M,10000,Paid,400.00,Everyone,lifestyle,"May 3, 2018",1.0.1,4.1 and up
5351,i am rich,lifestyle,3.8,3547,1.8M,100000,Paid,399.99,Everyone,lifestyle,"January 12, 2018",2.0,4.0.3 and up
5354,i am rich plus,family,4.0,856,8.7M,10000,Paid,399.99,Everyone,entertainment,"May 19, 2018",3.0,4.4 and up
5355,i am rich vip,lifestyle,3.8,411,2.6M,10000,Paid,299.99,Everyone,lifestyle,"July 21, 2018",1.1.1,4.3 and up
5356,i am rich premium,finance,4.1,1867,4.7M,50000,Paid,399.99,Everyone,finance,"November 12, 2017",1.6,4.0 and up
5357,i am extremely rich,lifestyle,2.9,41,2.9M,1000,Paid,379.99,Everyone,lifestyle,"July 1, 2018",1.0,4.0 and up


In [377]:
scam_apps = app_df.loc[app_df["Price"]>70.0].index
app_df.drop(index=scam_apps, axis=1, inplace=True)

### <span style="color:red">correlations</span>

### <span style="color:red">conclusion</span>

### <span style="color:red">Bonus: Python vs SQL</span>

In [376]:
@usd_decorator 

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_1404/966166846.py, line 1)